### Chatbot Prompt Engineering

Notebook for testing engineered prompts on LikeHome chatbot.

#### Enter Persona

Chatbot will use this for prompt to guide it through any prompt. See examples below. 

Add a new string to the `PERSONAS` list and run the remaining cells. Variables for `current_date` and `user_info` are also available to help user craft responses.

In [ ]:
from datetime import date

current_date = date.today()

# fake user details to test account retrieval ability
user_info = '''{
"name": "Bob Jones",
"reward_points": 50,
"email": bobjones@hotmail.com,
}'''

PERSONAS: list = [
# prompt 0
# does not work well when user asks question after confirming a hotel, chatbot marks confirmation still as true
# has trouble with date, can't find date when asked "book for next weekend" for example
f'''Your name is Nexus and you are tasked with answering user queries for a hotel booking website.
The date today is {current_date} for reference.

Users will prompt you with hotel information and you will parse their query to extract the information in the response. 
If they do not include enough information for you to determine a field's value default it to empty string.
In the response field create a message that asks for any remaining fields. 
If you have all the fields confirm with the user to start the search process, 
DO NOT mark user completion until you have received their explicit confirmation.
If they ask a unrelated question, politely respond shortly but remind them to confirm hotel search. 

Here is all the information on the users account if they ask about it: {user_info}''',

# prompt 1, works consistency with most situations
f'''Your name is Nexus and you are a chatbot for a hotel reservation website called LikeHome.
The date today is {current_date} for reference.

Respond to users if they ask about hotels or their account. Parse their queries to extract field details. 
In the response field create a message that asks for any fields you do not have enough information on. 
If you have all the fields confirm with the user to start adding hotels matching their criteria to their watchlist, 
DO NOT mark user completion until you have received their explicit confirmation.
MAKE SURE to reset user confirmation and hotel fields once you have received confirmation and added hotels to watchlist.
If they ask an unrelated question, politely respond shortly but remind them to confirm hotel search. 

Here is all the information on the users account if they ask about it: {user_info}''',

# TODO add additional prompts to test
]

#### Set-Up

Run the below cells to set up chat gpt requests.

In [ ]:
from openai import OpenAI
import numpy as np
import pandas as pd
from IPython.display import display
from pydantic import BaseModel
import json

convos = []
for persona in PERSONAS:
    convos.append([{"role": "system", "content": persona}])

client = OpenAI()

# define JSON format for model base outputs
class SearchFilter(BaseModel):
    location: str
    check_in: str
    check_out: str
    rooms: int
    adults: int
    children: int
    radius: int
    min_price: int
    max_price: int
    response: str
    user_search_confirmation: bool

def hotel_chat(messages: list):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        response_format=SearchFilter,
        temperature=0.2,
    )
    messages.append({"role": "assistant", "content": completion.choices[0].message.content})

    return json.dumps(json.loads(completion.choices[0].message.content), indent=4)

def main():
    global convos
    while (True):
        count = 0
        user_prompt = input("Your a prompt: ")
        if user_prompt == "quit":
            print("chat ended")
            break
        for convo in convos:
            convo.append({"role": "user", "content": user_prompt})
            print("-------------------------------")
            print("Using PERSONA at index %d" % count) 
            print(hotel_chat(convo))
            count += 1
        print(f'''-------------------------------\n''')

#### Run the chatbot

Run the below cell to test the chatbot. It will prompt u to enter a prompt. Enter **quit** to end the chat.

Chatbot will respond in following format:
```
{
    "location": "",
    "check_in": "",
    "check_out": "",
    "rooms": 0,
    "adults": 0,
    "children": 0,
    "radius": 0,
    "min_price": 0,
    "max_price": 0,
    "response": "response",
    "user_search_confirmation": false
}
```

look at the `response` field and evaluate how well it is doing, this is verbatim what the user will see when prompting the chatbot. Also ensure `user_confirmation` is only set to true when all fields look accurate. Once `user_confirmation` is true it will trigger creating a watchlist item in the actual app, so we want to make sure we don't do this prematurely.

In [ ]:
main()